In [1]:
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import argparse
import datetime
import requests
from lxml import html
import lxml.html as lh
from bs4 import BeautifulSoup
import warnings 
warnings.filterwarnings('ignore')

In [6]:
df = pd.read_csv('final_csv')

In [24]:
df['first_name'] = df['first_name'].str.lower()
df['last_name'] = df['last_name'].str.lower()
df['forb_name'] = df['first_name'] + '-' + df['last_name']
df['edu_forbes'] = 'None'
df


,Unnamed: 0,first_name,last_name,age,gender,country,position,real_time_position,worth(BUSD),worth(millions_USD),work_field,company,image,education,forb_name,edu_forbes
0,0,jeff,bezos,54,M,United States,1.0,1.0,112.0,0.000,Technology,Amazon,https://specials-images.forbesimg.com/imageser...,Yes,jeff-bezos,None
1,1,bill,gates,62,M,United States,2.0,2.0,90.0,-0.001,Technology,Microsoft,https://specials-images.forbesimg.com/imageser...,No,bill-gates,None
2,2,warren,buffett,87,M,United States,3.0,3.0,84.0,-0.002,Finance and Investments,Berkshire Hathaway,https://specials-images.forbesimg.com/imageser...,Yes,warren-buffett,None
3,3,bernard,arnault,69,M,France,4.0,4.0,72.0,0.000,Fashion & Retail,Lvmh,https://specials-images.forbesimg.com/imageser...,Yes,bernard-arnault,None
4,4,mark,zuckerberg,34,M,United States,5.0,5.0,71.0,0.000,Technology,Facebook,https://specials-images.forbesimg.com/imageser...,No,mark-zuckerberg,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,2203,zhao,xiaoqiang,51,M,China,2204.0,2134.0,1.0,0.000,Fashion & Retail,"Fashion, Entertainment",https://specials-images.forbesimg.com/imageser...,No,zhao-xiaoqiang,None
2204,2204,zhou,liangzhang,55,M,China,2205.0,2134.0,1.0,NaN,Manufacturing,Electrical Equipment,https://specials-images.forbesimg.com/imageser...,No,zhou-liangzhang,None
2205,2205,zhu,xingming,51,M,China,2206.0,1856.0,1.0,0.000,Manufacturing,Electrical Equipment,https://specials-images.forbesimg.com/imageser...,No,zhu-xingming,None
2206,2206,jun,zhuo,52,F,Hong Kong,2207.0,1978.0,1.0,0.000,Manufacturing,Printed Circuit Boards,https://specials-images.forbesimg.com/imageser...,No,jun-zhuo,None


In [50]:
cols = df['forb_name']

In [51]:
for i in cols:
    df.loc[i,'forb_name'] = re.sub(' & ', '-', df.loc[i,'forb_name'])

KeyError: 'jeff-bezos'

In [25]:
def education_list():
    a = list(df.loc[:, 'edu_forbes'] == 'None')
    count = -1
    none_education_list = []
    for i in a:
        count += 1
        if i == True:
            none_education_list.append(count)
    return none_education_list

In [41]:
for i in education_list():
    print(type(i))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

In [52]:
def web_scraping_education(df_col):
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    page_url = 'https://www.forbes.com/profile/'
    edu_url = []
    for i in education_list()[0:1000]:
        req = requests.get(page_url + df_col[i], headers=header)
        soup = BeautifulSoup(req.text, 'lxml')

        edu_find = soup.find('div', attrs={'class': 'profile-content'})
        edu_url.append(str(edu_find))
        
    edu_l = []
    for j in edu_url:
        edu_l.append(re.findall('Education', j))
    print(edu_l)
    edu = []
    for x in edu_l:
        if len(x) != 0:
            edu.append('Yes')
        else:
            edu.append('No')

    edu_df = pd.DataFrame(np.array(edu))
    edu_df.to_csv('edu_csv')

    return edu

In [53]:
web_scraping_education(df['forb_name'])

TypeError: can only concatenate str (not "float") to str

In [4]:
def web_scraping_education(df_col):
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    page_url = 'https://en.wikipedia.org/wiki/'
    edu_url=[]
    for i in none_education_list:
        req = requests.get(page_url+df_col[i], headers=header)
        soup = BeautifulSoup(req.text, 'lxml')

        edu_find = soup.find('table', attrs={'class': 'infobox biography vcard'})
        edu_url.append(str(edu_find))

    edu_l =[]
    for j in edu_url:
        edu_l.append(re.findall('Alma', j) or re.findall('Education', j))
    print(edu_l)
    edu = []
    for x in edu_l:
        if len(x) !=0:
            edu.append('Yes')
        else:
            edu.append('No')
    
    return edu

In [48]:
df['forb_name'][76]

'thomas & raymond-kwok'

In [5]:
education = web_scraping_education(df.name)

KeyboardInterrupt: 

In [ ]:
df['education'] = education
df

In [ ]:
df.drop('name', axis=1, inplace=True)

In [ ]:
df.to_csv('final_csv')
df

In [ ]:
# To replace the None values from colun country for the values of the list of countries that I just web scraped 
# from wikipedia first is necessary to convert 'None' to actually null values:
df['country'] = df.country.replace('None', np.nan).astype(object)

# The following command will give us the total of nulls in that specific column:
df.country.isnull().sum()

In [ ]:
# Since the command df.fillna doesn't accept lists as values, we need to transform it into a dictionary:
dictionary = dict(zip(none_country_list, country_name))
dictionary

In [ ]:
# Finally we can use the command fillna available in pandas to fill the nulls with respective values:
df.country.fillna(value=dictionary, inplace=True)

In [ ]:
# Since the synthase of the values from the web we scraped on wasn't the same we need to change it:
df['country'].replace('U.S', 'United States', inplace=True)
df['country'].replace('Union', 'Soviet Union', inplace=True)
df['country'].replace("People's Republic of China", 'China', inplace=True)
display(df)

In [ ]:
null_age = df[df['age'].isnull()].index.tolist()
len(null_age)

In [ ]:
# What I've seen is that all null age values exist because there're two names in these columns:
#for i in null_age:
#    df.loc[i,'name'] = re.sub(' & ', '_and_', df.loc[i,'name'])
#    df.loc[i,'name'] = re.sub(' ', '_', df.loc[i,'name'])

In [ ]:
# Tentar WEBSCRAPING POR GOOGLE COM O SIMBOLO &:
df.loc[54, 'age'] = 105
df.loc[76, 'age'] = 65
df.loc[89, 'age'] = 77
df.loc[129, 'age'] = 79
df.loc[141, 'age'] = 67
df.loc[223, 'age'] = 85
df.loc[224, 'age'] = 75
df.loc[243, 'age'] = 66
df.loc[253, 'age'] = 73
df.loc[268, 'age'] = 55
df.loc[295, 'age'] = 55
df.loc[326, 'age'] = 89
df.loc[357, 'age'] =Family_Kwee
df.loc[358, 'age'] = 40
df.loc[363, 'age'] =Wai_Ying_Lam
df.loc[405, 'age'] = 36
df.loc[414, 'age'] = 69
df.loc[421, 'age'] = 71
df.loc[455, 'age'] = 93 
df.loc[465, 'age'] = 27
df.loc[550, 'age'] = 59
df.loc[579, 'age'] = 68
df.loc[581,'age'] = 60
df.loc[596,'age'] = 65
df.loc[600,'age'] = 38
df.loc[645,'age'] = 55
df.loc[656,'age'] = Gabriele_Volkmann
df.loc[668,'age'] = 72
df.loc[694,'age'] = 70
df.loc[703,'age'] = 70
df.loc[731,'age'] = Abdulla_Al_Futtaim
df.loc[823,'age'] = 71
df.loc[889,'age'] = Jose_And_Francisco_Jose_Calderon_Rojas
df.loc[916,'age'] =
df.loc[1008,'age'] =
df.loc[1028,'age'] =
df.loc[1078,'age'] =
df.loc[1108,'age'] =
df.loc[1133,'age'] =
df.loc[1143,'age'] =
df.loc[1154,'age'] =
df.loc[1182,'age'] =
df.loc[1230,'age'] =
df.loc[1289,'age'] =
df.loc[1290,'age'] =
df.loc[1383,'age'] =
df.loc[1502,'age'] =
df.loc[1560,'age'] =
df.loc[1632,'age'] =
df.loc[1718,'age'] =
df.loc[1719,'age'] =
df.loc[1750,'age'] =
df.loc[1786,'age'] =
df.loc[1807,'age'] =
df.loc[1826,'age'] =Di_Lu
df.loc[1856,'age'] =Karin_Schick
df.loc[1867,'age'] =Meirong_Yang
df.loc[1868,'age'] = Fawzi_Al-Kharafi
df.loc[2010,'age'] = Mohannad_Al-Kharafi
df.loc[2019,'age'] = Manas_Chiaravanond
df.loc[2020,'age'] = Yupa_Chiaravanond
df.loc[2021,'age'] = 67
df.loc[2028, 'age'] = 56
df.loc[2044, 'age'] = 62
df.loc[2117, 'age'] = 38

In [ ]:
df.loc[268, 'name']

In [ ]:
for i in null_age:
    print(df.loc[i, 'name'])

In [ ]:
def web_scraping_bday(df_col):
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    page_url = 'https://en.wikipedia.org/wiki/'
    age_url=[]
    count=0
    for i in null_age[:5]:
        req = requests.get(page_url+df_col[i], headers=header)
        soup = BeautifulSoup(req.text, 'lxml')

        age_find = soup.find('span', attrs={'class': 'bday'})
        age_url.append(str(age_find))
        print(age_url)
    age_l=[]
    for j in age_url:
        age_l.append(re.findall('[0-9]+', j))
    print(type(age_url[1]))
    return age_l

In [ ]:
age = web_scraping_bday(df.name)
age

In [17]:
import glob
aa = glob.glob('../data/results/*.jpg')
aa

['../data/results/1Ray_Dalio.jpg', '../data/results/results2.jpg']

In [23]:
import re
for image in aa:
    print(re.match('../data/results/1\w*.jpg', image))

<re.Match object; span=(0, 30), match='../data/results/1Ray_Dalio.jpg'>
None


In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import argparse
import requests
from lxml import html
import lxml.html as lh
from bs4 import BeautifulSoup

df = pd.read_csv('../package1/df_cleaned')

a = list(df.loc[:, 'education'] == 'None')
count = -1
none_education_list = []
for i in a:
    count += 1
    if i == True:
        none_education_list.append(count)


def web_scraping_education(df_col):
    header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    page_url = 'https://en.wikipedia.org/wiki/'
    edu_url = []
    for i in none_education_list:
        req = requests.get(page_url + df_col[i], headers=header)
        soup = BeautifulSoup(req.text, 'lxml')

        edu_find = soup.find('table', attrs={'class': 'infobox biography vcard'})
        edu_url.append(str(edu_find))

    edu_l = []
    for j in edu_url:
        edu_l.append(re.findall('Alma', j) or re.findall('Education', j))

    edu = []
    for x in edu_l:
        if len(x) != 0:
            edu.append('Yes')
        else:
            edu.append('No')

def webscraping():

    df['education'] = web_scraping_education(df.name)

    df.drop('name', axis=1, inplace=True)
    print(df)
    #return df.to_csv('final_csv')

webscraping()